## Newton methods

#### Problem:
$$
f(\vec{x}) \rightarrow min,\\
f: \Omega \rightarrow \mathbb{R}, \\
\Omega \subset \mathbb{R^n}, f(\vec{x}) \mbox{ is convex}, \\
f(\vec{x}) \mbox{ - is twice diffirentiable on } \Omega\\
\vec{x_*} \in \Omega, f_{min} = f(\vec{x_*})
$$

We can greater efficency of finding $x_*$ if we use information not only about function gradient, but also Hessian $H(\vec{x})$

In simple variant on every k iteration function is approximated in neighborhood of point $\vec{x}_{k-1}$ by quadratic function $\phi_{k}(x)$ then $\vec{x}_k$ is found and the procedure continiues

By using Tailor series we can represent our function in neighborhood of point $x_{k}$ as
$$
f(\vec{x}) = f(\vec{x}_{k} + (\nabla f(\vec{x}_k), \vec{x} - \vec{x}_k) + \frac{1}{2}(H(\vec{x}_k)(\vec{x} - \vec{x}_k), \vec{x} - \vec{x}_k) + o(|\vec{x} - \vec{x}_k|)
$$

So our quadratic approximation $\phi_k(\vec{x})$ would be
$$
\phi_{k+1}(\vec{x}) = f(\vec{x}_{k} + (\nabla f(\vec{x}_k), \vec{x} - \vec{x}_k) + \frac{1}{2}(H(\vec{x}_k)(\vec{x} - \vec{x}_k), \vec{x} - \vec{x}_k)
$$

If our $H(\vec{x}_{k})$ is positive determined (function is convex), then $\vec{x}_{k+1}$ is single minimum of quadratic approximation and can be found using: 
$$ 
\nabla \phi_{k+1}(\vec{x}) = \nabla f(\vec{x}_k) + H(\vec{x}_k)(\vec{x} - \vec{x}_k) = \vec{0}
$$

Then we get
$$
\vec{x}_{k+1} = \vec{x}_{k}  - H^{-1}(\vec{x}_{k}) \nabla f(\vec{x}_{k})
$$

If our dimension number $n$ of space $\mathbb{R}$ is big enough, then finding $H^{-1}$ is very big problem. In this case it expedient to find minimum of $\phi_k(\vec{x})$ by using **gradient methods** or **conjugate directions method**
$\widetilde{\vec{x}}_k = argmin\{\phi_{k}(\vec{x})\}$ is just an approximation, using this we can build *relaxational sequence* 

$$
\vec{x}_{k} = \vec{x}_{k-1} + \lambda_k(\widetilde{\vec{x}}_{k} - \vec{x}_{k-1}) = \vec{x}_{k-1} + \lambda_k\vec{p}_{k} \\ 
\vec{p}_k = -H^{-1}(\vec{x}_{k-1}) \nabla f(\vec{x}_{k-1}) \mbox{ - direction of descent}
$$

We can find $\lambda_k$ different ways, for example find $argmin\{f(\vec{x}_{k-1} + \lambda_k\vec{p}_k\}$ or by method of step splitting


In [30]:
import matplotlib as mplib
import math as m
import numpy as np
from numpy.linalg import norm
from scipy import linalg
from scipy import sparse
from scipy import optimize
import matplotlib.pyplot as plt
from scipy.misc import derivative

from onedim_optimize import quadratic_approx, fibbonaci_method, middle_point_method, newton_modified, qubic_approx
from scipy.optimize import approx_fprime, line_search

from test_functions import rastrigin, ackley, sphere, beale, goldstein_price, booth, bukin, himmelblau, eggholder, cross

def der(f, x, n):
    return derivative(f, x, dx=np.float64(1e-7), n=n)

def upgraded_newton(f, a, b, epsilon):
    der1 = der(f, a, 1)
    sec_der = der(f, a, 2)
    if sec_der <= 0:
        print("Starting Fibbonaci")
        an, bn, k = fibbonaci_method(f, a, b, epsilon/100)
        if f((an+bn)/2) > f(b):
            return b, k+1
        else:
            return (an+bn)/2, k+1
    x0 = a
    x1 = x0 - np.divide(der1, sec_der)
    der2 = der(f, x1, 1)
    k = 0
    while(m.fabs(x0 - x1) > epsilon):
        if der2 - der1 == 0:
            an, bn, k = fibbonaci_method(f, x1, b, epsilon/100)
            return (an+bn)/2, k
        x2 = x1 - np.divide(x1 - x0, der2 - der1) * der2
        x0 = x1
        x1 = x2
        der1 = der2
        der2 = der(f, x1, 1)
        k += 1
    if x1 > b:
        x1 = b
    return x1, k 

def toOneParamFunc(f, x, w):
    return lambda p: f(x + p*w) 

def build_plot(f, rng, name=""):
    X = np.array([i * (rng[1] - rng[0])/100 + rng[0] for i in range(0, 101)])
    Y = np.array([f(x) for x in X])
    plt.suptitle(name)
    plt.plot(X, Y)
    return plt

def argmin(f, a, b, eps):
#     plt = build_plot(f, [a, b])
#     plt2 = build_plot(lambda x: der(f, x, 1), [a, b])
#     a, b, k = fibbonaci_method(f, a, b, eps)
#     K = k
#     x = (a+b)/2
    x, k =  upgraded_newton(f, a, b, eps)
    K = k
#     plt.scatter([x], (f(x)))
#     plt.show()
#     plt2.show()
    return x, K



def approx_gradient(f, eps):
    return lambda x: approx_fprime(x, f, eps)

def partial_der(f_dx, f_x, dx, j):
    p = np.divide(f_dx - f_x, dx)
    return p

def hessian_in_point(x, f, grad, eps):
     gr = grad(x)
     n = len(gr) 
     hes = []
     for i in range (0, n):
        x_delta = np.array(x[:])
        x_delta[i] = x_delta[i] + eps
        gr_delta = grad(x_delta)
        partials = np.array([partial_der(gr_delta[j], part, eps, j) for j,part in enumerate(gr)])
        hes.append(partials)
     return np.array(hes)

def hessian(f, grad, eps):
    return lambda x: hessian_in_point(x, f, grad, eps) 

### Test functions

In [32]:
rosenbrok = lambda x: (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2 
danilov = lambda x: x[0] + 2*x[1] + 4*m.sqrt(1 + x[0]**2 + x[1]**2)

# test1 = [
#     f1,
#     approx_gradient(f1, 1e-8),
#     np.array([-2, 1]),
#     0.01,
# ]
# test2 = [
#     f2,
#     approx_gradient(f2, 1e-8),
#     np.array([-1, -2]),
#     0.001,[ 99936.00079581 -69503.99452882]
# ]

test_danilov = [
    danilov,
    approx_gradient(danilov, 1e-8),
    np.array([10, 5]),
    1e-5,
]

test_rosen = [
    rosenbrok,
    approx_gradient(rosenbrok, np.float64(1e-8)),
    np.array([-2, 2]),
    0.001
]


test_rastrigin = [
    rastrigin,
    approx_gradient(rastrigin, np.float64(1e-10)),
    np.array([2, 1]),
    1e-4
]

test_ackley = [
    ackley,
    approx_gradient(ackley, np.float64(1e-10)),
    np.array([1, 1]),
    1e-4
]

test_sphere = [
    sphere,
    approx_gradient(sphere, np.float64(1e-10)),
    np.array([100, -255]),
    1e-5
]

test_beale = [
    beale,
    approx_gradient(beale, np.float64(1e-10)),
    np.array([2, 2]),
    1e-5
]

test_goldstein = [
    goldstein_price,
    approx_gradient(goldstein_price, np.float64(1e-9)),
    np.array([-3, -1]),
    1e-5
]

test_booth = [
    booth,
    approx_gradient(booth, np.float64(1e-8)),
    np.array([30, 50]),
    1e-5
]

test_bukin = [
    bukin,
    approx_gradient(bukin, np.float64(1e-10)),
    np.array([-10.5, 1.5]),
    1e-5
]

test_himmel = [
    himmelblau,
    approx_gradient(himmelblau, np.float64(1e-10)),
    np.array([56, 41]),
    1e-5
]

test_egg = [
    eggholder,
    approx_gradient(eggholder, np.float64(1e-10)),
    np.array([353, -200]),
    1e-7
]

test_cross = [
    cross,
    approx_gradient(cross, np.float64(1e-10)),
    np.array([2, -2]),
    1e-5
]

### Newton method

The common Newton method is to find our $H^{-1}$ matrix and than build relaxetion sequence by this rule:
$$
\vec{x}_{k+1} = \vec{x}_{k}  - H^{-1}(\vec{x}_{k}) \nabla f(\vec{x}_{k})
$$

But there is a problem with matrix $H$, it needs to be always positive determinated or $H^{-1}$ won't exist.

To solve this problem, let's check if $H$ is positive determinated, if not, then let's pick $\eta$_k, such that:
$$
\widetilde{H}_k = \eta_kI_n + H(\vec{x}_{k-1})
$$
$\widetilde{H}$ is positive determinated matrix, that we pick instead of $H$


In [3]:
def common_newton(f, gr, x, epsilon):
    hess = hessian(f2, gr, 1e-6)
    w = -gr(x)
    k = 0
    while(norm(w) > epsilon):
        H = hess(x)
        print(x, w)
        print(H)
        h = linalg.solve(H, w)
        x = x + h
        w = -gr(x)
        k += 1
    return f(x), x, k

In [4]:
f2 = lambda x: (x[0]**2 - x[1])**2 + (x[0] - 1)**2
danilov = lambda x: x[0] + 2*x[1] + 4*m.sqrt(1 + x[0]**2 + x[1]**2)
rosenbrok = lambda x: (1 - x[0])**2 + 100*(x[1] - x[0]**2)**2 


# test2 = [
#     f2,
#     approx_gradient(f2, 1e-8),
# #     hessian(f2, approx_gradient(f2, 1e-6), 1e-6),
#     np.array([-1, -2]),
#     0.001,
# ]

# test_danilov = [
#     danilov,
#     approx_gradient(danilov, np.float64(1e-8)),
# #     hessian(danilov, approx_gradient(danilov, np.float64(1e-6)), np.float64(1e-6)),
#     np.array([-2, -1]),
#     0.01,
# ]

# test_rosen = [
#     rosenbrok,
#     approx_gradient(rosenbrok, np.float64(1e-8)),
# #     hessian(rosenbrok,  approx_gradient(rosenbrok, np.float64(1e-8)), np.float64(1e-6)),
#     np.array([10, 20]),
#     np.float64(1e-4)
# ]

fmin, xmin, K = common_newton(*test2)
print(f"""
x minimum: {xmin},
f minimum: {fmin},
number of iterations: {K}
""")

NameError: name 'test2' is not defined

In [ ]:
def newton_upgraded(f, gr, x, epsilon):
    hess = hessian(f2, gr, 1e-6)
    w = -gr(x)
    phi = toOneParamFunc(f, x, w)
    k = 0
    n = 0
    while(norm(w) > epsilon):
        H = hess(x)
        print(x, w)
        print(H)
        h = linalg.solve(H, w)
        phi = toOneParamFunc(f, x, h)
        l, i = argmin(phi, 0, 1, epsilon) 
        n += i
        x = x + l * h
        w = -gr(x)
        k += 1
    return f(x), x, k, n

In [ ]:
f2 = lambda x: (x[0]**2 - x[1])**2 + (x[0] - 1)**2
danilov = lambda x: x[0] + 2*x[1] + 4*m.sqrt(1 + x[0]**2 + x[1]**2)

test2 = [
    f2,
    approx_gradient(f2, np.float64(1e-8)),
#     hessian(f2, approx_gradient(f2, np.float64(1e-6)), np.float64(1e-6)),
    np.array([-1, -2]),
    np.float64(1e-3),
]

test_danilov = [
    danilov,
    approx_gradient(danilov, np.float64(1e-8)),
#     hessian(danilov, approx_gradient(danilov, np.float64(1e-8)), np.float64(1e-8)),
    np.array([-1, -2]),
    0.01,
]

fmin, xmin, K, N = newton_upgraded(*test2)
print(f"""
x minimum: {xmin},
f minimum: {fmin},
number of iterations: {K},
number of one-dimension minimization iterations: {N}
""")

In [31]:
def count_next_matrix(A, dw,dx):
    y = dx
    z = A.dot(dw)
    first_part = np.dot(np.transpose([y]), [dx]).dot(np.divide(1, y.dot(dw)))
    sec_part =  np.dot(np.transpose([z]), [z]).dot(np.divide(1, z.dot(dw)))
    return A - first_part - sec_part

def count_next_matrix_2(A, dw,dx):
    I = np.identity(len(dx))
    r = np.divide(1, np.dot(dx, dw))
    C1 = (I - r*(np.dot(np.transpose([dw]), [dx])))
    C2 = (I - r*(np.dot(np.transpose([dx]), [dw])))
    first_part = C1.dot(A).dot(C2)
    sec_part =  r*np.dot(np.transpose([dx]), [dx])
    return first_part - sec_part

def DFP(f, grad, x, epsilon):
    w2 = -grad(x)
    phi = toOneParamFunc(f, x, w2)
    x1 = x
    k = 1
    fev, jev = 0, 1
    A = np.identity(len(x))
    l, f_lin, j_lin = argmin(phi, 0, 1, np.divide(epsilon, 1e3))
    fev += f_lin
    jev += j_lin
    print(x1, f(x1), l, w2)
    x2 = x1 + l * w2
    while(norm(w2) > epsilon):
        w1 = w2
        w2 = -grad(x2)
        jev += 1
        A = count_next_matrix_2(A, w2 - w1, x2 - x1)
        p = A.dot(w2)
        x1 = x2
        phi = toOneParamFunc(f, x2, p)
        l, f_lin, j_lin = argmin(phi, 0, 1, np.divide(epsilon, 1e3))
        print(x1, f(x1), l, p)
        x2 = x1 + l * p
        k += 1
        n += i
    return f(x2), x2, k, n
        

In [33]:
fmin, xmin, K, N = DFP(*test_booth)

print(f"""
x minimum: {xmin},
f minimum: {fmin},
number of iterations: {K},
number of one-dimension minimization iterations: {N}
""")

print(optimize.minimize(rosenbrok, np.array([-2, 2]), jac=approx_gradient(booth, np.float64(1e-8)), method='BFGS'))

ValueError: not enough values to unpack (expected 3, got 2)